In [1]:
# Primer paso: Explorar el dataset real
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
# Leyendo datos
print("Cargando dataset de Google Analytics...")
df = pd.read_parquet('../data/raw/data_sample.parquet')

Cargando dataset de Google Analytics...


In [15]:
print("=== INFORMACIÓN BÁSICA DEL DATASET ===")
print(f"Dimensiones: {df.shape}")
print(f"Columnas: {list(df.columns)}")
print(f"Total de columnas: {len(df.columns)}")
#print("\nTipos de datos:")
#print(df.dtypes)

=== INFORMACIÓN BÁSICA DEL DATASET ===
Dimensiones: (74457, 77)
Columnas: ['transaction_date', 'parsed_date', 'transaction_id', 'transaction_revenue_usd', 'transaction_tax_usd', 'transaction_shipping_usd', 'transaction_affiliation', 'currency_code', 'product_sku', 'product_name', 'product_category', 'product_brand', 'product_variant', 'product_quantity', 'product_price_usd', 'product_revenue_usd', 'is_impression', 'is_click', 'promo_id', 'promo_name', 'promo_creative', 'promo_position', 'visitor_id', 'session_id', 'session_number', 'session_start_time', 'total_visits', 'total_hits', 'total_pageviews', 'time_on_site_seconds', 'bounces', 'new_visits', 'traffic_source', 'traffic_medium', 'campaign', 'keyword', 'ad_content', 'referral_path', 'is_true_direct', 'channel_grouping', 'browser', 'browser_version', 'operating_system', 'os_version', 'is_mobile', 'mobile_device_brand', 'mobile_device_model', 'device_category', 'device_language', 'screen_resolution', 'continent', 'sub_continent', 'c

In [ ]:
df_10 = df[df.columns[:10]]
df_5 = df[df.columns[:5]]
print("\nPrimeras filas:")
print(df_10.head())


Primeras filas:
  transaction_date parsed_date   transaction_id  transaction_revenue_usd  \
0         20170801  2017-08-01  ORD201708011814                      NaN   
1         20170801  2017-08-01  ORD201708011814                    40.29   
2         20170801  2017-08-01  ORD201708011814                      NaN   
3         20170801  2017-08-01  ORD201708011814                    40.29   
4         20170801  2017-08-01  ORD201708011814                      NaN   

   transaction_tax_usd  transaction_shipping_usd   transaction_affiliation  \
0                  NaN                       NaN  Google Merchandise Store   
1                 8.82                      11.0  Google Merchandise Store   
2                  NaN                       NaN  Google Merchandise Store   
3                 8.82                      11.0  Google Merchandise Store   
4                  NaN                       NaN  Google Merchandise Store   

  currency_code     product_sku                          

In [29]:
print("\n=== ESTADÍSTICAS DESCRIPTIVAS ===")
print(df_5.describe(include='all'))


=== ESTADÍSTICAS DESCRIPTIVAS ===
       transaction_date parsed_date   transaction_id  transaction_revenue_usd  \
count             74457       74457            74457             36744.000000   
unique              365         365            11551                      NaN   
top            20161212  2016-12-12  ORD201703041515                      NaN   
freq                550         550              146                      NaN   
mean                NaN         NaN              NaN               287.388543   
std                 NaN         NaN              NaN              1028.358260   
min                 NaN         NaN              NaN                 1.200000   
25%                 NaN         NaN              NaN                46.960000   
50%                 NaN         NaN              NaN                96.900000   
75%                 NaN         NaN              NaN               230.870000   
max                 NaN         NaN              NaN             23952.560

In [30]:
print("\n=== VALORES NULOS ===")
null_analysis = df_10.isnull().sum()
print(null_analysis[null_analysis > 0])


=== VALORES NULOS ===
transaction_revenue_usd     37713
transaction_tax_usd         48876
transaction_shipping_usd    37877
dtype: int64


In [33]:
print("\n=== INFORMACIÓN DE TRANSACCIONES ===")
# Verificar columnas relacionadas con transacciones
transaction_cols = [col for col in df.columns if 'transact' in col.lower() or 'revenue' in col.lower()]
print("Columnas de transacción/revenue:", transaction_cols)

print("\n=== INFORMACIÓN DE PRODUCTOS ===")
product_cols = [col for col in df.columns if 'product' in col.lower() or 'sku' in col.lower()]
print("Columnas de producto:", product_cols)


=== INFORMACIÓN DE TRANSACCIONES ===
Columnas de transacción/revenue: ['transaction_date', 'transaction_id', 'transaction_revenue_usd', 'transaction_tax_usd', 'transaction_shipping_usd', 'transaction_affiliation', 'product_revenue_usd']

=== INFORMACIÓN DE PRODUCTOS ===
Columnas de producto: ['product_sku', 'product_name', 'product_category', 'product_brand', 'product_variant', 'product_quantity', 'product_price_usd', 'product_revenue_usd']
